In [ ]:
pip install requests msal jsons azure-storage-blob requests_toolbelt

In [ ]:
from services.aadservice import AadService
aad_token = AadService.get_access_token()

request_header =  {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + aad_token}

In [ ]:
workspace_id="16453fea-cb0f-4ef4-b1b2-bede9d3b92be"
display_name="AdventureWorks"

In [ ]:
import requests
from requests_toolbelt.multipart.encoder import MultipartEncoder

report_name = "AdventureWorks"

url = f'https://api.powerbi.com/v1.0/myorg/groups/{workspace_id}/imports?datasetDisplayName={report_name}'
headers = {
    'Content-Type': 'multipart/form-data',
    'authorization': 'Bearer ' + aad_token
}
file_location = 'files/AdventureWorks.pbix'
# you need this dictionary to convert a binary file into form-data format
# None here means we skip the filename and file content is important 
files = {'value': (None, open(file_location, 'rb'), 'multipart/form-data')}

mp_encoder = MultipartEncoder(fields=files)

r = requests.post(
    url=url,
    data=mp_encoder,  # The MultipartEncoder is posted as data, don't use files=...!
    # The MultipartEncoder provides the content-type header with the boundary:
    headers=headers
)
